In [1]:
########################################################
#
# import requirements
#
########################################################
import logging
import os
import librosa
import math
import os
import re

from random import shuffle

import numpy as np

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/Users/valerio/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/valerio/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/valerio/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/valerio/opt/anaconda3/lib/pytho

In [2]:
class FeaturesExtraction:

    "Music audio features for genre classification"
    hop_length = None
    genre_list = [
        "blues",
        "classical",
        "country",
        "disco",
        "hiphop",
        "jazz",
        "metal",
        "pop",
        "reggae",
        "rock",
        "bamileke",
        "bikutsi",
        "makossa",
        "salsa",
        "zouk"
    ]

    #ORIGINAL GTZAN AUDIO SAMPLES
    dir_blues = "./gtzan/gtzan+/gtzan/blues"
    dir_classical = "./gtzan/gtzan+/gtzan/classical"
    dir_country = "./gtzan/gtzan+/gtzan/country"
    dir_disco = "./gtzan/gtzan+/gtzan/disco"
    dir_hiphop = "./gtzan/gtzan+/gtzan/hiphop"
    dir_jazz = "./gtzan/gtzan+/gtzan/jazz"
    dir_metal = "./gtzan/gtzan+/gtzan/metal"
    dir_pop = "./gtzan/gtzan+/gtzan/pop"
    dir_reggae = "./gtzan/gtzan+/gtzan/reggae"
    dir_rock = "./gtzan/gtzan+/gtzan/rock"

    #AFRO EXTENDED GTZAN AUDIO SAMPLES
    dir_bamileke = "./gtzan/gtzan+/afro/bamileke"
    dir_bikutsi = "./gtzan/gtzan+/afro/bikutsi"
    dir_makossa = "./gtzan/gtzan+/afro/makossa"
    dir_salsa = "./gtzan/gtzan+/afro/salsa"
    dir_zouk = "./gtzan/gtzan+/afro/zouk"



    #dir_trainfolder = "./gtzan/_train"
    #dir_devfolder = "./gtzan/_validation"
    #dir_testfolder = "./gtzan/_test"
    #dir_all_files = "./gtzan"

    train_X_preprocessed_data = "./gtzan/data_train_input.npy"
    train_Y_preprocessed_data = "./gtzan/data_train_target.npy"
    dev_X_preprocessed_data = "./gtzan/data_validation_input.npy"
    dev_Y_preprocessed_data = "./gtzan/data_validation_target.npy"
    test_X_preprocessed_data = "./gtzan/data_test_input.npy"
    test_Y_preprocessed_data = "./gtzan/data_test_target.npy"

    train_X = train_Y = None
    dev_X = dev_Y = None
    test_X = test_Y = None

    def __init__(self):
        self.hop_length = 512

        self.timeseries_length_list = []
       # self.trainfiles_list = self.path_to_audiofiles(self.dir_trainfolder)
       # self.devfiles_list = self.path_to_audiofiles(self.dir_devfolder)
       # self.testfiles_list = self.path_to_audiofiles(self.dir_testfolder)

        self.all_files_list = []

        self.all_files_list.extend(self.path_to_audiofiles(self.dir_blues))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_classical))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_country))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_disco))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_hiphop))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_jazz))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_metal))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_pop))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_reggae))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_rock))

        self.all_files_list.extend(self.path_to_audiofiles(self.dir_bamileke))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_bikutsi))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_makossa))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_salsa))
        self.all_files_list.extend(self.path_to_audiofiles(self.dir_zouk))

        shuffle(self.all_files_list)

        self.trainfiles_list, self.testfiles_list, self.trainfiles_list, self.testfiles_list = train_test_split(self.all_files_list, self.all_files_list, test_size=0.2, random_state=1)
        self.trainfiles_list, self.devfiles_list, self.trainfiles_list, self.devfiles_list = train_test_split(self.trainfiles_list, self.trainfiles_list, test_size=0.25, random_state=1)

        
        self.all_files_list = []
        
        
        self.all_files_list.extend(self.trainfiles_list)
        self.all_files_list.extend(self.devfiles_list)
        self.all_files_list.extend(self.testfiles_list)
    
        self.timeseries_length = (
            128
        )   # sequence length == 128, default fftsize == 2048 & hop == 512 @ SR of 22050
        #  equals 128 overlapped windows that cover approx ~3.065 seconds of audio, which is a bit small!

    def load_preprocess_data(self):
        print("[DEBUG] total number of files: " + str(len(self.timeseries_length_list)))

        # Training set
        self.train_X, self.train_Y = self.extract_audio_features(self.trainfiles_list)
        
        print("\nTRAIN_X array:\n")
        print(self.train_X)
        
        print("\nTRAIN_Y array:\n")
        print(self.train_Y)
        
        
        with open(self.train_X_preprocessed_data, "wb") as f:
            np.save(f, self.train_X)
            
        with open(self.train_Y_preprocessed_data, "wb") as f:
            self.train_Y = self.one_hot(self.train_Y)
            np.save(f, self.train_Y)

        # Validation set
        self.dev_X, self.dev_Y = self.extract_audio_features(self.devfiles_list)
        with open(self.dev_X_preprocessed_data, "wb") as f:
            np.save(f, self.dev_X)
        with open(self.dev_Y_preprocessed_data, "wb") as f:
            self.dev_Y = self.one_hot(self.dev_Y)
            np.save(f, self.dev_Y)

        # Test set
        self.test_X, self.test_Y = self.extract_audio_features(self.testfiles_list)
        with open(self.test_X_preprocessed_data, "wb") as f:
            np.save(f, self.test_X)
        with open(self.test_Y_preprocessed_data, "wb") as f:
            self.test_Y = self.one_hot(self.test_Y)
            np.save(f, self.test_Y)

    def load_deserialize_data(self):

        self.train_X = np.load(self.train_X_preprocessed_data)
        self.train_Y = np.load(self.train_Y_preprocessed_data)

        self.dev_X = np.load(self.dev_X_preprocessed_data)
        self.dev_Y = np.load(self.dev_Y_preprocessed_data)

        self.test_X = np.load(self.test_X_preprocessed_data)
        self.test_Y = np.load(self.test_Y_preprocessed_data)

    def precompute_min_timeseries_len(self):
        for file in self.all_files_list:
            print("Loading " + str(file))
            y, sr = librosa.load(file)
            self.timeseries_length_list.append(math.ceil(len(y) / self.hop_length))

    def extract_audio_features(self, list_of_audiofiles):
        
        print(list_of_audiofiles)

        data = np.zeros(
            (len(list_of_audiofiles), self.timeseries_length, 33), dtype=np.float64
        )
        
        target = []

        for i, file in enumerate(list_of_audiofiles):
            print(file)
            
            y, sr = librosa.load(file)
            
            #MFCC selection
            mfcc = librosa.feature.mfcc(
                y=y, sr=sr, hop_length=self.hop_length, n_mfcc=13
            )
            
            #SPECTRAL_CENTER selection
            spectral_center = librosa.feature.spectral_centroid(
                y=y, sr=sr, hop_length=self.hop_length
            )
            
            #CHROMA selection
            chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=self.hop_length)
            
            #SPECTRAL CONTRAST selection
            spectral_contrast = librosa.feature.spectral_contrast(
                y=y, sr=sr, hop_length=self.hop_length
            )

            splits = re.split("[ .]", file)
            print(splits)
            genre = re.split("[ /]", splits[1])[4]
            print(genre)
            target.append(genre)

            # MFCC Extraction
            data[i, :, 0:13] = mfcc.T[0:self.timeseries_length, :]
            
            #SPECTRAL_CENTER Extraction
            data[i, :, 13:14] = spectral_center.T[0:self.timeseries_length, :]
            
            #CHROMA Extraction
            data[i, :, 14:26] = chroma.T[0:self.timeseries_length, :]
            
            #SPECTRAL_CONTRAST Extraction
            data[i, :, 26:33] = spectral_contrast.T[0:self.timeseries_length, :]

            print(
                "Extracted features audio track %i of %i."
                % (i + 1, len(list_of_audiofiles))
            )

        return data, np.expand_dims(np.asarray(target), axis=1)

    # ONE HOT ENCODING
    def one_hot(self, Y_genre_strings):
        y_one_hot = np.zeros((Y_genre_strings.shape[0], len(self.genre_list)))
        for i, genre_string in enumerate(Y_genre_strings):
            print(genre_string)
            index = self.genre_list.index(genre_string)
            y_one_hot[i, index] = 1
        return y_one_hot

    @staticmethod
    def path_to_audiofiles(dir_folder):
        list_of_audio = []
        for file in os.listdir(dir_folder):
            if file.endswith(".au") or file.endswith(".wav"):
                directory = "%s/%s" % (dir_folder, file)
                list_of_audio.append(directory)
        return list_of_audio


In [3]:
# set logging level
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [4]:
########################################################
#
# Check on pre-processed files. It generates those files and if they already exists use them.
#
########################################################
genre_features = FeaturesExtraction()

if (
    os.path.isfile(genre_features.train_X_preprocessed_data)
    and os.path.isfile(genre_features.train_Y_preprocessed_data)
    and os.path.isfile(genre_features.dev_X_preprocessed_data)
    and os.path.isfile(genre_features.dev_Y_preprocessed_data)
    and os.path.isfile(genre_features.test_X_preprocessed_data)
    and os.path.isfile(genre_features.test_Y_preprocessed_data)
):
    print("Preprocessed files exist, deserializing npy files")
    genre_features.load_deserialize_data()
else:
    print("Preprocessing raw audio files")
    genre_features.load_preprocess_data()

print("Training X shape: " + str(genre_features.train_X.shape))
print("Training Y shape: " + str(genre_features.train_Y.shape))
print("Dev X shape: " + str(genre_features.dev_X.shape))
print("Dev Y shape: " + str(genre_features.dev_Y.shape))
print("Test X shape: " + str(genre_features.test_X.shape))
print("Test Y shape: " + str(genre_features.test_Y.shape))

input_shape = (genre_features.train_X.shape[1], genre_features.train_X.shape[2])


Preprocessed files exist, deserializing npy files
Training X shape: (900, 128, 33)
Training Y shape: (900, 15)
Dev X shape: (300, 128, 33)
Dev Y shape: (300, 15)
Test X shape: (300, 128, 33)
Test Y shape: (300, 15)


In [5]:
########################################################
#
# Long short-term memory RNN using Adam optimization
#
########################################################

print("Build LSTM RNN model ...")
model = Sequential()

model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=input_shape))
model.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(units=genre_features.train_Y.shape[1], activation="softmax"))

print("Compiling model")
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

Build LSTM RNN model ...
Compiling model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 128)          82944     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_1 (Dense)              (None, 15)                495       
Total params: 104,047
Trainable params: 104,047
Non-trainable params: 0
_________________________________________________________________


In [ ]:
########################################################
#
# Training
# Batch size = 35
# Epochs = 400
#
########################################################

print("Training ...")
batch_size = 10
num_epochs = 500
model.fit(
    genre_features.train_X,
    genre_features.train_Y,
    batch_size=batch_size,
    epochs=num_epochs,
)

Training ...
Epoch 1/500
900/900 [==============================] - 29s 33ms/step - loss: 2.6893 - accuracy: 0.0878
Epoch 2/500
900/900 [==============================] - 28s 32ms/step - loss: 2.5455 - accuracy: 0.1600
Epoch 3/500
900/900 [==============================] - 26s 29ms/step - loss: 2.4583 - accuracy: 0.1756
Epoch 4/500
900/900 [==============================] - 20s 22ms/step - loss: 2.4294 - accuracy: 0.1811
Epoch 5/500
900/900 [==============================] - 20s 22ms/step - loss: 2.4670 - accuracy: 0.1689
Epoch 6/500
900/900 [==============================] - 20s 23ms/step - loss: 2.3907 - accuracy: 0.1922
Epoch 7/500
900/900 [==============================] - 20s 22ms/step - loss: 2.3407 - accuracy: 0.1956
Epoch 8/500
900/900 [==============================] - 19s 21ms/step - loss: 2.3152 - accuracy: 0.2000
Epoch 9/500
900/900 [==============================] - 19s 21ms/step - loss: 2.2841 - accuracy: 0.2111
Epoch 10/500
900/900 [==============================] - 19s 

900/900 [==============================] - 18s 20ms/step - loss: 1.4173 - accuracy: 0.5022
Epoch 80/500
900/900 [==============================] - 17s 19ms/step - loss: 1.3830 - accuracy: 0.5133
Epoch 81/500
900/900 [==============================] - 19s 21ms/step - loss: 1.3601 - accuracy: 0.5344
Epoch 82/500
900/900 [==============================] - 19s 21ms/step - loss: 1.3422 - accuracy: 0.5511
Epoch 83/500
900/900 [==============================] - 18s 21ms/step - loss: 1.2997 - accuracy: 0.5489
Epoch 84/500
900/900 [==============================] - 19s 21ms/step - loss: 1.3101 - accuracy: 0.5367
Epoch 85/500
900/900 [==============================] - 17s 19ms/step - loss: 1.3248 - accuracy: 0.5411
Epoch 86/500
900/900 [==============================] - 19s 21ms/step - loss: 1.3219 - accuracy: 0.5411
Epoch 87/500
900/900 [==============================] - 16s 18ms/step - loss: 1.3212 - accuracy: 0.5500
Epoch 88/500
900/900 [==============================] - 16s 18ms/step - loss:

900/900 [==============================] - 17s 19ms/step - loss: 1.0632 - accuracy: 0.6300
Epoch 158/500
900/900 [==============================] - 17s 19ms/step - loss: 1.0558 - accuracy: 0.6389
Epoch 159/500
900/900 [==============================] - 16s 18ms/step - loss: 1.0598 - accuracy: 0.6222
Epoch 160/500
900/900 [==============================] - 17s 19ms/step - loss: 1.0482 - accuracy: 0.6411
Epoch 161/500
900/900 [==============================] - 16s 18ms/step - loss: 1.0188 - accuracy: 0.6433
Epoch 162/500
900/900 [==============================] - 16s 18ms/step - loss: 1.0421 - accuracy: 0.6411
Epoch 163/500
900/900 [==============================] - 16s 18ms/step - loss: 0.9944 - accuracy: 0.6411
Epoch 164/500
900/900 [==============================] - 16s 18ms/step - loss: 1.0385 - accuracy: 0.6567
Epoch 165/500
900/900 [==============================] - 16s 18ms/step - loss: 1.0117 - accuracy: 0.6500
Epoch 166/500
900/900 [==============================] - 18s 20ms/ste

In [ ]:
########################################################
#
# Validation
#
########################################################

print("Validating ...")
score, accuracy = model.evaluate(
    genre_features.dev_X, genre_features.dev_Y, batch_size=batch_size, verbose=1
)
print("Dev loss:  ", score)
print("Dev accuracy:  ", accuracy)


In [ ]:
########################################################
#
# Testing
#
########################################################

print("Testing ...")
score, accuracy = model.evaluate(
    genre_features.test_X, genre_features.test_Y, batch_size=batch_size, verbose=1
)
print("Test loss:  ", score)
print("Test accuracy:  ", accuracy)

In [ ]:
########################################################
#
# Store HDF5 file model
#
########################################################

model_filename = "lstm_genre_classifier_lstm.h5"
print("Saving model: " + model_filename)
model.save(model_filename)    

In [ ]:
########################################################
#
# Prediction test
#
########################################################
import sys

def load_model(model_path, weights_path):
    "Load the trained LSTM model from directory for genre classification"
    with open(model_path, "r") as model_file:
        trained_model = model_from_json(model_file.read())
    trained_model.load_weights(weights_path)
    trained_model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return trained_model


def extract_audio_features(file):
    "Extract audio features from an audio file for genre classification"
    timeseries_length = 128
    features = np.zeros((1, timeseries_length, 33), dtype=np.float64)

    y, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=512, n_mfcc=13)
    spectral_center = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=512)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=512)

    features[0, :, 0:13] = mfcc.T[0:timeseries_length, :]
    features[0, :, 13:14] = spectral_center.T[0:timeseries_length, :]
    features[0, :, 14:26] = chroma.T[0:timeseries_length, :]
    features[0, :, 26:33] = spectral_contrast.T[0:timeseries_length, :]
    return features


def get_genre(model, music_path):
    "Predict genre of music using a trained model"
    prediction = model.predict(extract_audio_features(music_path))
    predict_genre = FeaturesExtraction().genre_list[np.argmax(prediction)]
    return predict_genre


if __name__ == "__main__":
    #PATH = sys.argv[1] if len(sys.argv) == 2 else "./audio/classical_music.mp3"
    PATH = "./audio/"
    f = []
    for (dirpath, dirnames, filenames) in os.walk(PATH):
        f.extend(filenames)
        break
    
    for f in filenames:
        print("Trying to predict the genre of "+f)
        GENRE = get_genre(model, PATH+f)
        print("Model predict: {}".format(GENRE))
    